<a href="https://colab.research.google.com/github/dkurbatovv/Python/blob/main/Neural_Network_predict_price_house.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot

from folium import plugins

In [ ]:
df = pd.read_csv('kc_house_data.csv')

In [ ]:
df.head()

In [ ]:
df = df.drop('id', axis = 1)

In [ ]:
df.describe()

In [ ]:
g1 = [go.Box(y=df.bedrooms,name="Bedrooms",marker=dict(color="rgba(51,0,0,0.9)"),hoverinfo="name+y")]
g2 = [go.Box(y=df.bathrooms,name="Bathrooms",marker=dict(color="rgba(0,102,102,0.9)"),hoverinfo="name+y")]
g3 = [go.Box(y=df.floors,name="Floors",marker=dict(color="rgba(204,0,102,0.9)"),hoverinfo="name+y")]
layout2 = go.Layout(title="Bedrooms,Bathrooms and Floors",yaxis=dict(range=[0,13])) 
fig2 = go.Figure(data=g1+g2+g3,layout=layout2)
iplot(fig2)

In [ ]:
plt.figure(figsize=(12,10))
g = sns.countplot(df['bedrooms'])

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(df.corr(), annot = True, cmap="icefire")

In [ ]:
sns.scatterplot(x = 'sqft_living', y = 'price', data = df)

In [ ]:
df.hist(bins = 20, figsize = (20,20), color = 'blue');

In [ ]:
df["color"] = ""
df.color[df.grade == 1] = "rgb(255,255,255)"
df.color[df.grade == 2] = "rgb(220,220,220)"
df.color[df.grade == 3] = "rgb(242, 177, 172)"
df.color[df.grade == 4] = "rgb(255,133,27)"
df.color[df.grade == 5] = "rgb(255,255,204)"
df.color[df.grade == 6] = "rgb(255,65,54)"
df.color[df.grade == 7] = "rgb(178,37,188)"
df.color[df.grade == 8] = "rgb(51,51,0)"
df.color[df.grade == 9] = "rgb(37,188,127)"
df.color[df.grade == 10] = "rgb(26,51,176)"
df.color[df.grade == 11] = "rgb(132,10,10)"
df.color[df.grade == 12] = "rgb(82,80,80)"
df.color[df.grade == 13] = "rgb(0,0,0)"

dataplus = df[np.logical_and(df.grade >= 7,df.yr_built >= 2000)] 
lats = list(dataplus.lat.values)
longs = list(dataplus.long.values)

mapbox_access_token = 'pk.eyJ1IjoiZGFya2NvcmUiLCJhIjoiY2pscGFheHA1MXdqdjNwbmR3c290MTZ6dCJ9.K1FMv_q3ZVlKP13RrjFkjg'
mapp = [go.Scattermapbox(lat=lats,lon=longs,mode="markers",marker=dict(size=4,color=dataplus["color"]) ,hoverinfo="text",text="Grade:"+dataplus.grade.apply(str)+" Built Year:"+dataplus.yr_built.apply(str)+" Price:"+dataplus.price.apply(str))]
layout5 = dict(title="Grade(+7) - Built Year(+2000) Map",width=800,height=750,hovermode="closest",mapbox=dict(bearing=0,pitch=0,zoom=9,center=dict(lat=47.5,lon=-122.161),accesstoken=mapbox_access_token))
fig5 = go.Figure(data=mapp,layout=layout5)
iplot(fig5);

In [ ]:
selected_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'sqft_above', 'sqft_basement']

In [ ]:
X = df[selected_features]

In [ ]:
X.head()


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
X

In [ ]:
y = df['price']

In [ ]:
y.head()

In [ ]:
y = y.values.reshape(-1,1)

In [ ]:
y_scaler = scaler.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_scaler, test_size = 0.2, random_state = 42 )

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()

model.add(Dense(units = 100, activation = 'relu', input_shape=(7, )))
model.add(Dense(units = 100, activation = 'relu'))
model.add(Dense(units = 100, activation = 'relu'))
model.add(Dense(units = 1, activation = 'linear'))

In [ ]:
model.compile(optimizer='Adam', loss='mean_squared_error')

In [ ]:
history = model.fit(X_train, y_train, batch_size = 64, epochs = 20, validation_split = 0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss Progress During Training')
plt.ylabel('Training and Validation Loss')
plt.xlabel('Epoch number')
plt.legend(['Training Loss', 'Validation Loss']);

In [ ]:
y_predict = model.predict(X_test)
plt.plot(y_test, y_predict, "^", color = 'r')
plt.xlabel("Model Predictions")
plt.ylabel("True Value (ground Truth)")
plt.title('Linear Regression Predictions')
plt.show()